In [1]:
import torch

## 1. Prepare dataset

#### Tensor(类) 可以存标量，向量，矩阵，高维数据， 包含两个重要成员data， grad
#### data 用来保存权重w本身的值
#### grad 用来保存损失函数对权重的导数

In [2]:
x_data = torch.Tensor([[1.0], [2.0], [3.0]]) 
y_data = torch.Tensor([[2.0], [4.0], [6.0]])

## 2. Design model using Class

### class torch.nn.Linear(in_features, out_features, bias=True)
#### 可以完成: y = Ax + b 的线性方程
#### in_features,  out_features: 表示输入输出样本是几维的. (输入输出样本的维度必须相同)
#### bias: 是否需要偏置量

In [3]:
class LinearModel(torch.nn.Module):           # 继承父类 nn.Module
    def __init__(self):                       # 初始化时执行的函数
        super(LinearModel, self).__init__()   # 调用父类的构造 
        self.linear = torch.nn.Linear(1, 1)   # 构造Linear对象(这里面就包含了权重w和偏置b), 可以自动完成
                                              # x * w + b 的计算.       (1, 1)表示输入输出是1维
    
    def forward(self, x):                     # 覆盖父类中__call__()中的forward函数
        y_pred = self.linear(x)               # 执行前馈过程中的计算 
        return y_pred

In [4]:
model = LinearModel() # 实例化模型 --相当于Java中的new一个对象, 
                      # model也是一个callable的,可以直接被调用, model(x)就是执行LinearModel中的forward(self, x).

## 3. Construct loss and optimizer

### 3.1 construct loss:
### class torch.nn.MSELoss(size_average = True, reduce = True)
#### size_average = False 得到整个batch所有像素loss和， 默认为True得到的是平均到像素的loss
#### reduce: 是否求和降维(一般不考虑)

In [5]:
criterion = torch.nn.MSELoss(size_average=False) # 传入y_pred 和 y_data 就可以求出损失函数 loss

/home/zgy/anaconda3/lib/python3.8/site-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


### 3.2 construct optimizer
### class torch.optim.SGD(params, lr=< object object >, momentum=0, dampening=0, weight_decay=0, nesterov=False)   
#### model父类中有成员函数:parameters,可以检查model中所有成员, 如果成员里有相应的权重, 就把这些权重都加到要训练的结果上

In [6]:
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01) 
# SGD优化器：随机梯度下降，对应权重更新公式: w = w -  a(loss 对于 w 的导数)

## 4. Training cycle 
### 前面的和下面的操作都是在构建计算图! 而不是运算. 第四步可以概括为三个过程: 前馈,反馈, 更新.
### 通过计算图得到的结果需要调用 .data 得到一个Tensor类型的数据或者调用 .item() 得到一个标量.

In [7]:
for epoch in range(1000):
    y_pred = model(x_data)             # 1.算y_pred
    loss = criterion(y_pred, y_data)   # 2.算loss
    print(epoch, loss.item())          # 1.和 2.是前馈的过程
    
    optimizer.zero_grad()              # 反向传播前, 梯度清零.
    loss.backward()                    # 3.backward: 反向传播 得到所有要更新权重(这里只有w)的梯度 w.grad 
                                       #  (w.grad 这里是一个Tensor,为了避免构建计算图需要使用 w.grad.data 进行更新)
    optimizer.step()                   # 4.更新权重w 进行: w = w -  lr * grad 的运算
 
print("w = ", model.linear.weight.item())  # .item()可以理解为得到weight的数值, 因为不加item()得到的是一个矩阵[[]].
print("b = ", model.linear.bias.item())

0 148.81640625
1 66.27688598632812
2 29.532228469848633
3 13.174159049987793
4 5.891617774963379
5 2.6492526531219482
6 1.2054636478424072
7 0.562354564666748
8 0.2756907343864441
9 0.14771181344985962
10 0.0903799831867218
11 0.06450377404689789
12 0.0526355504989624
13 0.0470087043941021
14 0.04416469484567642
15 0.04256470501422882
16 0.04152325168251991
17 0.04073529690504074
18 0.04006470367312431
19 0.03945104405283928
20 0.03886709362268448
21 0.038301050662994385
22 0.03774727135896683
23 0.03720329329371452
24 0.036667969077825546
25 0.036140624433755875
26 0.03562110662460327
27 0.03510919213294983
28 0.034604545682668686
29 0.034107234328985214
30 0.0336170494556427
31 0.0331338569521904
32 0.03265770524740219
33 0.03218831494450569
34 0.03172585368156433
35 0.031269799917936325
36 0.030820470303297043
37 0.030377592891454697
38 0.029940929263830185
39 0.029510673135519028
40 0.029086526483297348
41 0.0286684799939394
42 0.02825644239783287
43 0.027850434184074402
44 0.02745

455 7.156183710321784e-05
456 7.053358422126621e-05
457 6.951906834729016e-05
458 6.851960642961785e-05
459 6.7537410359364e-05
460 6.656609912170097e-05
461 6.560842302860692e-05
462 6.466516788350418e-05
463 6.373616633936763e-05
464 6.282034155447036e-05
465 6.191663123900071e-05
466 6.102400948293507e-05
467 6.01522988290526e-05
468 5.9286416217219085e-05
469 5.843258259119466e-05
470 5.7595534599386156e-05
471 5.676759610651061e-05
472 5.5948261433513835e-05
473 5.514523945748806e-05
474 5.435488492366858e-05
475 5.357235932024196e-05
476 5.280269397189841e-05
477 5.2044473704881966e-05
478 5.1295060984557495e-05
479 5.055976362200454e-05
480 4.983306280337274e-05
481 4.911529322271235e-05
482 4.841202826355584e-05
483 4.7712230298202485e-05
484 4.702789738075808e-05
485 4.635404184227809e-05
486 4.568780423142016e-05
487 4.5028733438812196e-05
488 4.438064570422284e-05
489 4.374686977826059e-05
490 4.3116895540151745e-05
491 4.24945137638133e-05
492 4.1884180973283947e-05
493 4.1

In [8]:
x_test = torch.Tensor([4.0])
y_test = model(x_test)
print("y_pred = ", y_test.data)

y_pred =  tensor([7.9998])
